## Import torch and model

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

In [3]:
import sys
sys.path.append("../common")

from model_generator import ModelGenerator
from net import Net

## Set hyper params

In [4]:
batch_size = 64
num_epoch  = 300

lr = 0.1
gamma = 0.2             # learning rate decay
weight_decay = 0.0001

## for SGD
opt_momentum = 0.9
opt_nesterov = True

dropout_on = False
batchnorm_on = True 

scheduler_step_size = [150, 225]

pretrained_model       = './cifar10_densenet100_pretrained.pth'

## Load dataset

In [5]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010))])

transform_train = transforms.Compose(
    [transforms.RandomHorizontalFlip(),
     transforms.RandomCrop(32, 4),
     transforms.ToTensor(),
     transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


## Define densenet 100

In [6]:
model_gen = ModelGenerator(dropout = dropout_on, batchnorm = batchnorm_on)

model_gen.CifarDensenetConfig(k = 12, num_layers = 100, cifar = 10)
model = model_gen.GetCifarDensenet()

net = Net(model)

net.Gpu()

In [7]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(net.GetTotalParams(), lr=lr, weight_decay=weight_decay, momentum=opt_momentum, nesterov=opt_nesterov )

scheduler = lr_scheduler.MultiStepLR(optimizer, milestones = scheduler_step_size, gamma = gamma)

## Training (back-propagation)

In [8]:
for epoch in range(num_epoch):  # loop over the dataset multiple times

    running_loss = 0.0
    scheduler.step()
    net.TrainMode()
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # wrap them in Variable
        inputs = Variable(inputs.cuda())
        labels = Variable(labels.cuda())

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        #scheduler.step()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss = (running_loss * i + loss.cpu().data.numpy()) / (i+1)

    correct = 0
    total = 0
    net.TestMode()
    for data in testloader:
        images, labels = data
        outputs = net(Variable(images.cuda()))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels.cuda()).sum()
        
    print('%d epoch end, loss: %3.6f, Test Acc: %4.2f %%' %(epoch + 1, running_loss, 100 * correct / total))
    
print('\nTraining is finished!')

1 epoch end, loss: 1.389578, Test Acc: 60.13 %
2 epoch end, loss: 0.867029, Test Acc: 66.03 %
3 epoch end, loss: 0.684609, Test Acc: 73.19 %
4 epoch end, loss: 0.582835, Test Acc: 79.06 %
5 epoch end, loss: 0.529145, Test Acc: 78.93 %
6 epoch end, loss: 0.487295, Test Acc: 81.49 %
7 epoch end, loss: 0.456000, Test Acc: 82.60 %
8 epoch end, loss: 0.434502, Test Acc: 82.15 %
9 epoch end, loss: 0.416055, Test Acc: 84.02 %
10 epoch end, loss: 0.395450, Test Acc: 83.91 %
11 epoch end, loss: 0.387572, Test Acc: 85.10 %
12 epoch end, loss: 0.370715, Test Acc: 84.26 %
13 epoch end, loss: 0.365019, Test Acc: 85.99 %
14 epoch end, loss: 0.354776, Test Acc: 85.48 %
15 epoch end, loss: 0.346449, Test Acc: 82.81 %
16 epoch end, loss: 0.340789, Test Acc: 83.80 %
17 epoch end, loss: 0.334127, Test Acc: 79.47 %
18 epoch end, loss: 0.327496, Test Acc: 84.46 %
19 epoch end, loss: 0.320897, Test Acc: 84.96 %
20 epoch end, loss: 0.313461, Test Acc: 83.26 %
21 epoch end, loss: 0.308879, Test Acc: 84.60 %
2

171 epoch end, loss: 0.024509, Test Acc: 93.75 %
172 epoch end, loss: 0.023579, Test Acc: 93.75 %
173 epoch end, loss: 0.023717, Test Acc: 94.16 %
174 epoch end, loss: 0.024998, Test Acc: 93.87 %
175 epoch end, loss: 0.028205, Test Acc: 93.36 %
176 epoch end, loss: 0.031664, Test Acc: 93.28 %
177 epoch end, loss: 0.034763, Test Acc: 93.43 %
178 epoch end, loss: 0.029694, Test Acc: 92.72 %
179 epoch end, loss: 0.030958, Test Acc: 93.78 %
180 epoch end, loss: 0.036106, Test Acc: 93.13 %
181 epoch end, loss: 0.033048, Test Acc: 93.43 %
182 epoch end, loss: 0.034024, Test Acc: 93.42 %
183 epoch end, loss: 0.038320, Test Acc: 93.35 %
184 epoch end, loss: 0.037832, Test Acc: 93.44 %
185 epoch end, loss: 0.038151, Test Acc: 92.71 %
186 epoch end, loss: 0.036169, Test Acc: 93.34 %
187 epoch end, loss: 0.040683, Test Acc: 93.30 %
188 epoch end, loss: 0.042282, Test Acc: 93.08 %
189 epoch end, loss: 0.038516, Test Acc: 93.26 %
190 epoch end, loss: 0.043921, Test Acc: 93.08 %
191 epoch end, loss:

## Fine-tuning (KD + Cross-entropy)

In [9]:
correct = 0
total = 0
net.TestMode()
for data in testloader:
    images, labels = data
    outputs = net(Variable(images.cuda()))
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()

print('Accuracy of the network on the 10000 test images: %4.2f %%' % (100 * correct / total))

Accuracy of the network on the 10000 test images: 95.20 %


In [10]:
torch.save(net.GetStateDict(), pretrained_model)

In [11]:
net.blocks

[[Conv2d(3, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
  'ConvBlock'],
 [[BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True),
   ReLU(inplace),
   Conv2d(24, 48, kernel_size=(1, 1), stride=(1, 1), bias=False),
   BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True),
   ReLU(inplace),
   Conv2d(48, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)],
  [BatchNorm2d(36, eps=1e-05, momentum=0.1, affine=True),
   ReLU(inplace),
   Conv2d(36, 48, kernel_size=(1, 1), stride=(1, 1), bias=False),
   BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True),
   ReLU(inplace),
   Conv2d(48, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)],
  [BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True),
   ReLU(inplace),
   Conv2d(48, 48, kernel_size=(1, 1), stride=(1, 1), bias=False),
   BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True),
   ReLU(inplace),
   Conv2d(48, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)],
  [BatchNorm2